In [1]:
# =============================================================================
# 小売業需要予測モデル - データクレンジング改善記録
# =============================================================================
# 
# 【実施した施策】
# - n_estimatorsを100に設定（ベースライン）
# - データクレンジングのみ：欠損値と外れ値の除去
# - シンプルなアプローチで過学習を回避
# 
# 【スコア】
# - ベースライン（線形回帰）: 3.9937572546850784
# - 前回の結果（rf100）: 3.07256739424164
# - 前回の結果（master）: 3.644648（悪化）
# - 今回の結果: [提出後に記録]
# 
# 【考察】
# - [提出後に記録]
# 
# 【今後の改善案】
# - データクレンジングの効果を確認
# - 段階的な特徴量追加
# - ハイパーパラメータの調整
# 
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats
import os

In [2]:
base_dir = '../data/'
sales_df = pd.read_csv(base_dir + 'sales_history.csv')
item_categories_df = pd.read_csv(base_dir + 'item_categories.csv')
category_names_df = pd.read_csv(base_dir + 'category_names.csv')
test_df = pd.read_csv(base_dir + 'test.csv')
submission_df = pd.read_csv(base_dir + 'sample_submission.csv', header=None)

In [3]:
# 日付から年と月を抽出
sales_df['年'] = sales_df['日付'].apply(lambda x: x.split('-')[0])
sales_df['月'] = sales_df['日付'].apply(lambda x: x.split('-')[1])

# =============================================================================
# データクレンジング（シンプル版）
# =============================================================================

print("データクレンジング前の形状:", sales_df.shape)

# 1. 欠損値の確認と除去
print("\n欠損値の確認:")
print(sales_df.isnull().sum())

# 欠損値がある行を除去
sales_df = sales_df.dropna()
print("欠損値除去後の形状:", sales_df.shape)

# 2. 外れ値の検出と除去（Z-score法）
z_scores = np.abs(stats.zscore(sales_df['売上個数']))
outlier_threshold = 3  # Z-score > 3 を外れ値とする

print(f"\n外れ値の数: {np.sum(z_scores > outlier_threshold)}")
print(f"外れ値の割合: {np.sum(z_scores > outlier_threshold) / len(sales_df) * 100:.2f}%")

# 外れ値を除去
sales_df = sales_df[z_scores <= outlier_threshold]
print("外れ値除去後の形状:", sales_df.shape)

# 3. 売上個数が0以下の異常値を除去
sales_df = sales_df[sales_df['売上個数'] > 0]
print("異常値除去後の形状:", sales_df.shape)

print(f"\nデータクレンジング完了！")
print(f"最終的なデータ形状: {sales_df.shape}")
print(f"除去されたデータ数: {492614 - sales_df.shape[0]}")
print(f"データ保持率: {sales_df.shape[0] / 492614 * 100:.2f}%")

# 月ごとの売上個数を集計
sales_month_df = sales_df.groupby(['商品ID', '店舗ID', '年', '月'])['売上個数'].sum().reset_index()

# 商品カテゴリIDを結合
train_df = pd.merge(sales_month_df, item_categories_df, on='商品ID', how='left')

# データ型を変換
train_df['年'] = train_df['年'].astype(int)
train_df['月'] = train_df['月'].astype(int)

# test_dfにも年と月を追加し、商品カテゴリIDを結合
test_df['年'] = 2022
test_df['月'] = 12
test_df = pd.merge(test_df, item_categories_df, on='商品ID', how='left')

# 特徴量とターゲット変数を定義
feature_columns = ['店舗ID', '商品ID', '年', '月', '商品カテゴリID']
target_column = '売上個数'

# 学習データとテストデータに分割
X_train = train_df[feature_columns]
y_train = train_df[target_column]
X_test = test_df[feature_columns]

データクレンジング前の形状: (1110198, 7)

欠損値の確認:
日付      0
店舗ID    0
商品ID    0
商品価格    0
売上個数    0
年       0
月       0
dtype: int64
欠損値除去後の形状: (1110198, 7)

外れ値の数: 10908
外れ値の割合: 0.98%
外れ値除去後の形状: (1099290, 7)
異常値除去後の形状: (1096692, 7)

データクレンジング完了！
最終的なデータ形状: (1096692, 7)
除去されたデータ数: -604078
データ保持率: 222.63%


In [4]:
# =============================================================================
# モデルの学習（n_estimators=100、シンプルなアプローチ）
# =============================================================================

# モデルの定義と学習（RandomForestRegressorを使用）
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 予測結果に負の値があれば0に変換
y_pred[y_pred < 0] = 0

print("モデルの学習が完了しました")
print(f"予測結果の形状: {y_pred.shape}")
print(f"予測値の範囲: {y_pred.min():.2f} ～ {y_pred.max():.2f}")

# 特徴量重要度の表示
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n特徴量重要度（上位5位）:")
print(feature_importance.head(5))

モデルの学習が完了しました
予測結果の形状: (3060,)
予測値の範囲: 1.00 ～ 20.76

特徴量重要度（上位5位）:
    feature  importance
1      商品ID    0.566143
0      店舗ID    0.235302
3         月    0.168293
2         年    0.029014
4  商品カテゴリID    0.001248


In [5]:
# =============================================================================
# 提出ファイルの作成
# =============================================================================

# submission_df の 1 列に予測結果 (y_pred) を代入
submission_df[1] = y_pred

# 提出ファイルの保存
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%m%d-%H%M")  # MMdd-HHmm形式
improvement_name = "clean"
submission_file_name = f'../submissions/20250909_Exercises3_Challenge_{improvement_name}_{timestamp}.csv'
submission_df.to_csv(submission_file_name, index=False, header=False)
print(f"提出ファイルを保存しました: {submission_file_name}")

# 現在の日時を取得してタイムスタンプ付きファイル名を作成
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%m%d-%H%M")  # MMdd-HHmm形式

# 提出用ファイル名（改善内容とタイムスタンプ付き）
# 注意: 新しい改善を試す際は、以下の改善内容部分を変更してください
improvement_name = "rf100"  # 例: "rf200", "xgb", "features", "ensemble" など
submission_file_name = f'../submissions/20250909_Exercises3_Challenge_{improvement_name}_{timestamp}.csv'

# 提出用データフレームをCSVファイルとして出力
submission_df.to_csv(submission_file_name, index=False, header=False)

print(f"\n提出用ファイル '{submission_file_name}' が作成されました。")
print("このファイルをコンペティションサイトに提出してください。")

提出ファイルを保存しました: ../submissions/20250909_Exercises3_Challenge_clean_0909-1447.csv

提出用ファイル '../submissions/20250909_Exercises3_Challenge_rf100_0909-1447.csv' が作成されました。
このファイルをコンペティションサイトに提出してください。


In [6]:
# =============================================================================
# スコア記録（提出後に値を入力）
# =============================================================================

from score_analysis import record_score, compare_scores, generate_improvement_summary

# 提出後にスコアを入力してください
current_score = None  # 提出後に実際のスコアを入力
model_name = "RandomForestRegressor (n_estimators=100) + データクレンジング"
features_used = "店舗ID, 商品ID, 年, 月, 商品カテゴリID"
notes = "データクレンジングのみ：欠損値・外れ値・異常値の除去"

if current_score is not None:
    # スコアを記録・分析
    result = record_score(
        current_score=current_score,
        model_name=model_name,
        features_used=features_used,
        notes=notes
    )
else:
    print("提出後にスコアを記録してください")
    print("例: current_score = 3.500000")
    print("スコアを入力後、このセルを再実行してください")


提出後にスコアを記録してください
例: current_score = 3.500000
スコアを入力後、このセルを再実行してください


In [8]:
# =============================================================================
# スコア記録・分析セル
# =============================================================================
# 提出後にスコアを記録してください

# スコア記録・分析ライブラリをインポート
from score_analysis import record_score, compare_scores, generate_improvement_summary

# スコア記録（提出後に値を入力）
current_score = 3.644648498790873  # 提出後にスコアを入力
model_name = "RandomForestRegressor (n_estimators=100)"
features_used = "店舗ID, 商品ID, 年, 月, 商品カテゴリID"
notes = "線形回帰からRandomForestに変更"

if current_score is not None:
    # スコアを記録・分析
    result = record_score(
        current_score=current_score,
        model_name=model_name,
        features_used=features_used,
        notes=notes
    )
else:
    print("スコアを記録してください。")
    print("例: current_score = 3.500000")
    print("その後、このセルを再実行してください。")


📊 スコア分析結果
前回のスコア: 3.644648
今回のスコア: 3.644648
改善幅: 0.000000
改善率: 0.00%
ベースラインからの改善率: 8.74%
使用モデル: RandomForestRegressor (n_estimators=100)
使用特徴量: 店舗ID, 商品ID, 年, 月, 商品カテゴリID
メモ: 線形回帰からRandomForestに変更
------------------------------------------------------------
➖ スコアに変化はありませんでした
